In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SLDE /GEEAQCSABLKNTTOGAA)RLS ( NM', 'calculationPrice': 'close', 'open': 137.81, 'openTime': 1651563260532, 'openSource': 'fliifaco', 'close': 140.47, 'closeTime': 1620347438193, 'closeSource': 'oiiclffa', 'high': 139.68, 'highTime': 1619678180873, 'highSource': 'TILtX aadeE rs', 'low': 134.95, 'lowTime': 1687808407880, 'lowSource': 'e e5ltncddra iypie1 mue', 'latestPrice': 141.41, 'latestSource': 'Close', 'latestTime': 'February 12, 2021', 'latestUpdate': 1675694401715, 'latestVolume': 61503629, 'iexRealtimePrice': 137.9, 'iexRealtimeSize': 1, 'iexLastUpdated': 1619532858698, 'delayedPrice': 139.26, 'delayedPriceTime': 1645766868073, 'oddLotDelayedPrice': 141.7, 'oddLotDelayedPriceTime': 1632759657319, 'extendedPrice': 140.34, 'extendedChange': 0.06, 'extendedChangePercent': 0.00045, 'extendedPriceTime': 1628272322087, 'previousClose': 137.83, 'previousVolume': 65000772, 'change': 0.25, 'changePercent': 0.00187, 'volum

In [7]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

38.06